In [ ]:
LORA_RANK=64
LORA_DROPOUT=0.2
ALL_TARGET_MODULES='qkvogud'
RSLORA=True
MODEL_SEED=42
EVAL_STEPS=200
BATCH_SIZE=8
GRADIENT_ACCUMULATION=1
EPOCH=5
TRAINING_SEED=3407
WEIGHT_DECAY=1.0

In [10]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-4B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...

==((====))==  Unsloth 2025.5.3: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.749 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!



Loading checkpoint shards:   0%|                                                                    | 0/2 [00:00<?, ?it/s]


Loading checkpoint shards:  50%|██████████████████████████████                              | 1/2 [00:01<00:01,  1.23s/it]


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.03it/s]


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.01s/it]

In [11]:
FONT_TO_MODULE = {
    "q": "q_proj",
    "k": "k_proj",
    "v": "v_proj",
    "o": "o_proj",
    "g": "gate_proj",
    "u": "up_proj",
    "d": "down_proj"
}

# Example: ALL_TARGET_MODULES = "gqk"
target_modules = [FONT_TO_MODULE[char] for char in ALL_TARGET_MODULES if char in FONT_TO_MODULE]

# If you want default behavior (all modules) when empty
if not target_modules:
    target_modules = list(FONT_TO_MODULE.values())

In [ ]:
lora_rank = LORA_RANK

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = target_modules,
    lora_alpha = lora_rank,
    lora_dropout = LORA_DROPOUT, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = MODEL_SEED,
    use_rslora = RSLORA,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

model_setup = f"""lora_rank = {LORA_RANK}

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = {target_modules},
    lora_alpha = lora_rank,
    lora_dropout = {LORA_DROPOUT}, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = {MODEL_SEED},
    use_rslora = {RSLORA},  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)"""

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth 2025.5.3 patched 36 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

# Define the paths to your dataset files
data_files = {
    'train': 'vuln_data_train_with_recom_CoT.csv',
    'val': 'vuln_data_val_with_recom_CoT.csv',
    'test': 'vuln_data_test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

In [ ]:
import random
from datasets import concatenate_datasets, DatasetDict

random.seed(42)

SYSTEM_PROMPT = [
    """You are the smartest AI solidity smart contract security auditor in the world""",
    """You are the greatest AI assistant smart contract security auditor in the world""",
    """You are the best solidity smart contract security auditor in the world""",
    """You are the greatest AI assistant solidity security researcher in the world""",
    """You are the best AI solidity smart contract security auditor in the world"""
]

PROMPTS = [
    """The given Solidity function is vulnerable and there was an explanation about the vulnerability. Give your patch recommendation based on that vulnerability.
    
    This is the vulnerable functions:
    ```solidity
    {code}
    ```
    
This is the vulnerability explanation: 
{explanation}

Recommendation:
""",

    """The following Solidity function contains security vulnerabilities. Based on vulnerability explanation give the recommendation to patch the security issues.
    
    Functions that has vulnerability:
    ```solidity
    {code}
    ```
    
Vulnerability explanation of the function: 
{explanation}

Recommendation:
""",

    """The provided Solidity function has security vulnerabilities. Your task is to give recommendation to close the security issue.
    
    Vulnerable code:
    ```solidity
    {code}
    ```
    
Vulnerability explanation: 
{explanation}

Recommendation:
""",

    """The Solidity function below is vulnerable. Give your best recommendation to close that security issue based on vulnerability explanation.
    
    Functions that vulnerable:
    ```solidity
    {code}
    ```
    
Vulnerability explanation: 
{explanation}

Recommendation:
""",

    """Analyze the following Solidity function. It has security vulnerabilities that need to be patched. Give your best recommendation.
    
    Vulnerable functions:
    ```solidity
    {code}
    ```
    
The vulnerability explanation: 
{explanation}

Recommendation:
""",
]

def make_conversational(examples, reasoning_idx, system_prompt, prompt):
    code = examples['vuln_code']
    desc = examples['vuln_explanation']
    recom = examples['vuln_recommendation']
    reasoning_content = examples[f'CoT_{reasoning_idx}']
    selected_system_prompt = system_prompt
    selected_user_prompt = prompt
    conversation = [{"role": "system", "content": selected_system_prompt}, 
                    {"role": "user", "content": selected_user_prompt.format(code=code, explanation=desc)},
                    {"role": "assistant", "reasoning_content":reasoning_content, "content": recom}]
    
    return { "conversations" : conversation}

# List of splits you want to process
splits = ['train', 'val', 'test']

# Dictionary to store concatenated datasets for each split
concatenated_datasets = {split: [] for split in splits}

# Process each dataset
for i in range(5):
    system_prompt = SYSTEM_PROMPT[i]
    prompt = PROMPTS[i]
    
    # Create a new dataset using map
    new_dataset = dataset.map(lambda ex: make_conversational(ex, i, system_prompt, prompt))
    
    # Append the correct split of the new dataset to the corresponding list
    for split in splits:
        concatenated_datasets[split].append(new_dataset[split])

# Now concatenate all datasets for each split
dataset = DatasetDict({
    split: concatenate_datasets(concatenated_datasets[split]).shuffle(seed=42) for split in splits
})

dataset = dataset.remove_columns(["vuln_title", "vuln_explanation", "severity", "vuln_recommendation"])

In [ ]:
from unsloth.chat_templates import get_chat_template

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(
        convo, 
        tokenize = False, 
        add_generation_prompt = False,
        enable_thinking=False) for convo in convos]
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [16]:
dataset['train'][5]["conversations"]

[{'content': 'You are the greatest AI assistant solidity security researcher in the world',
  'reasoning_content': None,
  'role': 'system'},
 {'content': 'The Solidity function below is vulnerable. Give your best recommendation to close that security issue based on vulnerability explanation.\n    \n    Functions that vulnerable:\n    ```solidity\n    function verify(\n        address signer,\n        MetaTransaction memory metaTx,\n        bytes32 sigR,\n        bytes32 sigS,\n        uint8 sigV\n    ) internal view returns (bool) {\n        require(signer != address(0), "NativeMetaTransaction: INVALID_SIGNER");\n        return\n            signer ==\n            ecrecover(\n                toTypedMessageHash(hashMetaTransaction(metaTx)),\n                sigV,\n                sigR,\n                sigS\n            );\n    }\n    ```\n    \nVulnerability explanation: \nThe `verify` function of `NativeMetaTransaction` calls the Solidity `ecrecover` function directly to verify the gi

In [17]:
dataset['train'][5]["text"]

'<|im_start|>system\nYou are the greatest AI assistant solidity security researcher in the world<|im_end|>\n<|im_start|>user\nThe Solidity function below is vulnerable. Give your best recommendation to close that security issue based on vulnerability explanation.\n    \n    Functions that vulnerable:\n    ```solidity\n    function verify(\n        address signer,\n        MetaTransaction memory metaTx,\n        bytes32 sigR,\n        bytes32 sigS,\n        uint8 sigV\n    ) internal view returns (bool) {\n        require(signer != address(0), "NativeMetaTransaction: INVALID_SIGNER");\n        return\n            signer ==\n            ecrecover(\n                toTypedMessageHash(hashMetaTransaction(metaTx)),\n                sigV,\n                sigR,\n                sigS\n            );\n    }\n    ```\n    \nVulnerability explanation: \nThe `verify` function of `NativeMetaTransaction` calls the Solidity `ecrecover` function directly to verify the given signature. However, the `e

In [18]:
tokenized = dataset['train'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
# print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Train Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['train'] = dataset['train'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Train Max Length =  1740


In [19]:
tokenized = dataset['val'].map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["conversations"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Val Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset['val'] = dataset['val'].select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Val Max Length =  1855


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

train_dataset = dataset['train']
val_dataset = dataset['val']

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False,
    args = TrainingArguments(
        eval_strategy="steps",
        eval_steps=EVAL_STEPS,
        save_steps=EVAL_STEPS,
        per_device_train_batch_size = BATCH_SIZE,
        per_device_eval_batch_size = BATCH_SIZE,
        gradient_accumulation_steps = GRADIENT_ACCUMULATION, 
        warmup_ratio = 0.1,
        num_train_epochs = EPOCH,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", 
        weight_decay = WEIGHT_DECAY,
        lr_scheduler_type = "cosine",
        seed = TRAINING_SEED,
        load_best_model_at_end=True,
        output_dir = "outputs_recommendation",
        report_to = "none",
    ),
)

training_setup = f"""from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

train_dataset = dataset['train']
val_dataset = dataset['val']

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        eval_strategy="steps",
        eval_steps={EVAL_STEPS},
        save_steps={EVAL_STEPS},
#         per_device_train_batch_size = 1,
        per_device_train_batch_size = {BATCH_SIZE},
        per_device_eval_batch_size = {BATCH_SIZE},
        gradient_accumulation_steps = {GRADIENT_ACCUMULATION}, # Fixed major bug in latest Unsloth
#         warmup_steps = 5,
        warmup_ratio = 0.1,
        num_train_epochs = {EPOCH}, # Set this for 1 full training run.
#         max_steps = 30,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Save more memory
        weight_decay = {WEIGHT_DECAY},
        lr_scheduler_type = "cosine",
        seed = {TRAINING_SEED},
        load_best_model_at_end=True,
        output_dir = "outputs_vuln_exp_v2",
        report_to = "none", # Use this for WandB etc
    ),
)"""

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [21]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

In [22]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|im_start|>system\nYou are the greatest AI assistant solidity security researcher in the world<|im_end|>\n<|im_start|>user\nThe Solidity function below is vulnerable. Give your best recommendation to close that security issue based on vulnerability explanation.\n    \n    Functions that vulnerable:\n    ```solidity\n    function verify(\n        address signer,\n        MetaTransaction memory metaTx,\n        bytes32 sigR,\n        bytes32 sigS,\n        uint8 sigV\n    ) internal view returns (bool) {\n        require(signer != address(0), "NativeMetaTransaction: INVALID_SIGNER");\n        return\n            signer ==\n            ecrecover(\n                toTypedMessageHash(hashMetaTransaction(metaTx)),\n                sigV,\n                sigR,\n                sigS\n            );\n    }\n    ```\n    \nVulnerability explanation: \nThe `verify` function of `NativeMetaTransaction` calls the Solidity `ecrecover` function directly to verify the given signature. However, the `e

In [23]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                                                                                                                                                                                                 <think>\nOkay let's see, the code is a verify function in a Solidity contract that checks if a signature is valid by using the ecrecover function. The vulnerability explanation says that ecrecover is susceptible to replay attacks because it allows malleable signatures. Even though the contract uses a nonce that's only used once, rejecting malleable signatures is still a best practice. The recommendation is to use the recover function from OpenZeppelin's ECDSA library instead. So the mitigation would involve replacing the direct call to ecrecover with the recover function from the ECDSA library, which is designed to handle these issues more securely. That should prevent replay attacks by properly rejecting malleable signatures.\n</th

In [24]:
tokenizer.decode(trainer.eval_dataset[5]["input_ids"])

'<|im_start|>system\nYou are the best AI solidity smart contract security auditor in the world<|im_end|>\n<|im_start|>user\nAnalyze the following Solidity function. It has security vulnerabilities that need to be patched. Give your best recommendation.\n    \n    Vulnerable functions:\n    ```solidity\n    function getLatestPrice(address num, address den) external view override returns (uint256, uint256) {\n        uint256 _price;\n        uint256 _decimals;\n        (_price, _decimals) = getChainlinkLatestPrice(num, den);\n        if (_decimals != 0) {\n            return (_price, _decimals);\n        }\n        (_price, _decimals) = getUniswapLatestPrice(num, den);\n        if (_decimals != 0) {\n            return (_price, _decimals);\n        }\n        revert("PriceOracle::getLatestPrice - Price Feed doesn\'t exist");\n    }\n    ```\n    \nThe vulnerability explanation: \nIf for whatever reason the Chainlink oracle returns a malformed price due to oracle manipulation or a malfunc

In [25]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.eval_dataset[5]["labels"]])

"                                                                                                                                                                                                                                                                                                                                 <think>\nOkay, let's check the functionality of the `getLatestPrice` function. It seems to be designed to fetch the latest price between two tokens using either Chainlink or Uniswap as a fallback. The function first tries Chainlink, and if that returns a non-zero `_decimals`, it returns that price. Otherwise, it tries Uniswap. If both fail, it reverts.\n\nHmm, this function seems suspicious in how it handles the price data from Chainlink. The code assumes that if `_decimals` is non-zero, the price is valid. But what if Chainlink returns a completely incorrect or manipulated price? The function would just pass that along without any checks.\n\nWait a second, if this vari

In [ ]:
from unsloth import unsloth_train

trainer_stats = unsloth_train(trainer)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,979 | Num Epochs = 5 | Total steps = 1,865
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 132,120,576/4,154,588,672 (3.18% trained)


Step,Training Loss,Validation Loss
200,0.762000,0.696095
400,0.532600,0.701568
600,0.734100,0.715334
800,0.408100,0.772534
1000,0.412800,0.785603
1200,0.280800,0.874038
1400,0.262000,0.879373
1600,0.203800,0.938340
1800,0.227000,0.939147


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
model.save_pretrained("ours_recom")  # Local saving
tokenizer.save_pretrained("ours_recom")

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("ours_recom/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())